In [ ]:
import glob
import os
import pathlib
import cv2
import numpy as np
import re
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [ ]:
def recreate_image_from_patches(patches, img_size, overlap, patch_size=256):
    
    reconstructed_image = np.zeros((img_size, img_size), dtype=np.float32) 
    patch_count = np.zeros((img_size, img_size), dtype=np.float32)

    step = patch_size - overlap
    idx = 0
    
    for y in range(0, img_size - patch_size + 1, step):
        for x in range(0, img_size - patch_size + 1, step):
            reconstructed_image[y:y + patch_size, x:x + patch_size] += patches[idx]
            patch_count[y:y + patch_size, x:x + patch_size] += 1
            idx += 1
    
    # Avoid division by zero
    patch_count[patch_count == 0] = 1e-10
    
    reconstructed_image //= patch_count  # Averaging the pixel values    
    return reconstructed_image

In [ ]:
def reconstruct(folder_path, original_filename, tile_size, tag):
    # Reconstructs an image from its tiles.   
    new_folder = Path(folder_path)
    new_folder = os.path.join(new_folder.parent.absolute(),'denoised_images',tag)
    os.makedirs(new_folder,exist_ok=True)
    tiles = []

    img_size=512
    overlap=128
    patch_size=tile_size
    
    # Setting the number of patches based on size
    number_of_patches = ((img_size - overlap) // (patch_size - overlap))**2
    
    # Sort based on tile number to propperly assign patch locations
    files = natural_sort(os.listdir(folder_path))
    
    # Sort tiles based on their filename to maintain order
    for filename in files:
        if os.path.splitext(original_filename)[0] in filename and tag in filename:
            tile_path = os.path.join(folder_path, filename)
            tile = cv2.imread(tile_path)
            tile = tile[:,:,0]
            tiles.append(tile)
    
    reconstructed_image = recreate_image_from_patches(tiles, img_size, overlap)
   
    # Save the reconstructed image
    output_image_path = os.path.join(new_folder, f'{original_filename}')
    print(output_image_path)
    cv2.imwrite(output_image_path, reconstructed_image)

In [ ]:
# Set the folder path where images are located and the tile size
path = pathlib.Path(os.getcwd())
FMD_testmix = path/'FMD_testmix'
tile_size = 256

In [ ]:
# Process images of various noise levels 
noise_levels = ['raw','avg2','avg4','avg8','avg16']
for i in noise_levels:
    print(f'working on {i} images')
    fn = sorted(os.listdir(FMD_testmix/i))
    for j in fn:
        reconstruct(FMD_testmix/'FMD_pred_tiles',j,tile_size,i)